Improvements:
* [X] Reverse input. Added in `def tokenize_en(text):`
* [X] Train on smaller dataset (currently using just 32k/2M pairs) (lower epoch instead). New trainset with 64k/160k-pairs and 2k5 most freq words from original dataset
* [ ] Change init model to normal distribution from $-0.8$ to $0.8$

In [ ]:
# https://github.com/bentrevett/pytorch-seq2seq/blob/master/4%20-%20Packed%20Padded%20Sequences%2C%20Masking%2C%20Inference%20and%20BLEU.ipynb
# based on https://gmihaila.github.io/tutorial_notebooks/pytorchtext_bucketiterator/#dataset-class

# Setup

In [1]:
!pip3 install torch==1.8.2 torchvision==0.9.2 torchaudio==0.8.2 --extra-index-url https://download.pytorch.org/whl/lts/1.8/cu111 -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 GB 908.7 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.5/17.5 MB 38.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 4.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchtext 0.14.1 requires torch==1.13.1, but you have torch 1.8.2+cu111 which is incompatible.


In [2]:
import torch
print(torch.__version__)

1.8.2+cu111


In [3]:
!pip install torchtext==0.9 -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 58.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 735.5/735.5 MB 2.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.9.2+cu111 requires torch==1.8.2, but you have torch 1.8.0 which is incompatible.
torchaudio 0.8.2 requires torch==1.8.2, but you have torch 1.8.0 which is incompatible.


In [4]:
!python -m spacy download en_core_web_sm -q
# !python -m spacy download de_core_news_sm -q
!python -m spacy download fr_core_news_sm -q

2023-02-26 03:14:59.585501: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-26 03:15:04.599362: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-02-26 03:15:04.599784: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-02-26 03:15:04.600366: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT War

# Work

In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from torchtext.legacy.datasets import Multi30k
from torchtext.legacy.data import Field, BucketIterator
from torchtext.legacy.data import Dataset, Example

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

import spacy
import numpy as np

import random
import math
import time
import pickle
from tqdm import tqdm

In [ ]:
# SEED = 1234
# random.seed(SEED)
# np.random.seed(SEED)
# torch.manual_seed(SEED)
# torch.cuda.manual_seed(SEED)
# torch.backends.cudnn.deterministic = True

In [6]:
# spacy_de = spacy.load('de_core_news_sm')
# spacy_fr = spacy.load('fr_core_news_sm')
# spacy_en = spacy.load('en_core_web_sm')

In [7]:
# def tokenize_de(text):
#     return [tok.text for tok in spacy_de.tokenizer(text)]
# def tokenize_fr(text):
#     return [tok.text for tok in spacy_fr.tokenizer(text)]
# def tokenize_en(text):
#     return [tok.text for tok in spacy_en.tokenizer(text)][::-1]

## Tutorial

In [ ]:
SRC = Field(tokenize = tokenize_de, 
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = True, 
            include_lengths = True)

TRG = Field(tokenize = tokenize_en, 
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = True)

In [ ]:
train_data, valid_data, test_data = Multi30k.splits(exts = ('.de', '.en'), 
                                                    fields = (SRC, TRG))

downloading training.tar.gz


training.tar.gz: 100%|██████████| 1.21M/1.21M [00:02<00:00, 422kB/s]


downloading validation.tar.gz


validation.tar.gz: 100%|██████████| 46.3k/46.3k [00:00<00:00, 113kB/s] 


downloading mmt_task1_test2016.tar.gz


mmt_task1_test2016.tar.gz: 100%|██████████| 66.2k/66.2k [00:00<00:00, 108kB/s] 


In [ ]:
len(train_data)

29000

In [ ]:
SRC.build_vocab(train_data, min_freq = 2)
TRG.build_vocab(train_data, min_freq = 2)

In [ ]:
BATCH_SIZE = 128

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_data, valid_data, test_data),
     batch_size = BATCH_SIZE,
     sort_within_batch = True,
     sort_key = lambda x : len(x.src),
     device = device)

In [ ]:
for i, batch in enumerate(train_iterator):
  break

In [ ]:
src, src_len = batch.src
src

tensor([[   2,    2,    2,    2],
        [   5,   18,    5,    8],
        [  25,  103,  496,   36],
        [ 124,  274,   10,   22],
        [ 646,   25,    5,   54],
        [   7,   11,  558,   69],
        [ 172,  827,  738,   12],
        [5072,   53,   11,   14],
        [  42,   12,   18,  247],
        [  33,    6,    0,   12],
        [  51,   50,   57,   10],
        [1356, 1816,  675,  284],
        [   4,    4,    4,    4],
        [   3,    3,    3,    3]], device='cuda:0')

In [ ]:
packed = nn.utils.rnn.pack_padded_sequence(src, src_len.to('cpu'))
packed

PackedSequence(data=tensor([   2,    2,    2,    2,    5,   18,    5,    5,   13,   30,  164,  618,
          10,   10,   31,    7,    5,    8,    7,  661,   70,   16,    6,  998,
          26,  141, 2329,   68,  983,  133,   10,   19,    7,    7,   38,    0,
           6,    6,   11,  100,  268, 1019,  111,  877,    4,    4,    4,    4,
           3,    3,    3,    3], device='cuda:0'), batch_sizes=tensor([4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4]), sorted_indices=None, unsorted_indices=None)

In [ ]:
outputs, lens = nn.utils.rnn.pad_packed_sequence(packed)
outputs, lens

(tensor([[   2,    2,    2,    2],
         [   5,   18,    5,    5],
         [  13,   30,  164,  618],
         [  10,   10,   31,    7],
         [   5,    8,    7,  661],
         [  70,   16,    6,  998],
         [  26,  141, 2329,   68],
         [ 983,  133,   10,   19],
         [   7,    7,   38,    0],
         [   6,    6,   11,  100],
         [ 268, 1019,  111,  877],
         [   4,    4,    4,    4],
         [   3,    3,    3,    3]], device='cuda:0'), tensor([13, 13, 13, 13]))

### Model

In [ ]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, enc_hid_dim, dec_hid_dim, dropout):
        super().__init__()
        
        self.embedding = nn.Embedding(input_dim, emb_dim)
        
        self.rnn = nn.GRU(emb_dim, enc_hid_dim, bidirectional = True)
        
        self.fc = nn.Linear(enc_hid_dim * 2, dec_hid_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src, src_len):
        
        #src = [src len, batch size]
        #src_len = [batch size]
        
        embedded = self.dropout(self.embedding(src))
        
        #embedded = [src len, batch size, emb dim]
                
        #need to explicitly put lengths on cpu!
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, src_len.to('cpu'))
                
        packed_outputs, hidden = self.rnn(packed_embedded)
                                 
        #packed_outputs is a packed sequence containing all hidden states
        #hidden is now from the final non-padded element in the batch
            
        outputs, _ = nn.utils.rnn.pad_packed_sequence(packed_outputs) 
            
        #outputs is now a non-packed sequence, all hidden states obtained
        #  when the input is a pad token are all zeros
            
        #outputs = [src len, batch size, hid dim * num directions]
        #hidden = [n layers * num directions, batch size, hid dim]
        
        #hidden is stacked [forward_1, backward_1, forward_2, backward_2, ...]
        #outputs are always from the last layer
        
        #hidden [-2, :, : ] is the last of the forwards RNN 
        #hidden [-1, :, : ] is the last of the backwards RNN
        
        #initial decoder hidden is final hidden state of the forwards and backwards 
        #  encoder RNNs fed through a linear layer
        hidden = torch.tanh(self.fc(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1)))
        
        #outputs = [src len, batch size, enc hid dim * 2]
        #hidden = [batch size, dec hid dim]
        
        return outputs, hidden

In [ ]:
class Attention(nn.Module):
    def __init__(self, enc_hid_dim, dec_hid_dim):
        super().__init__()
        
        self.attn = nn.Linear((enc_hid_dim * 2) + dec_hid_dim, dec_hid_dim)
        self.v = nn.Linear(dec_hid_dim, 1, bias = False)
        
    def forward(self, hidden, encoder_outputs, mask):
        
        #hidden = [batch size, dec hid dim]
        #encoder_outputs = [src len, batch size, enc hid dim * 2]
        
        batch_size = encoder_outputs.shape[1]
        src_len = encoder_outputs.shape[0]
        
        #repeat decoder hidden state src_len times
        hidden = hidden.unsqueeze(1).repeat(1, src_len, 1)
  
        encoder_outputs = encoder_outputs.permute(1, 0, 2)
        
        #hidden = [batch size, src len, dec hid dim]
        #encoder_outputs = [batch size, src len, enc hid dim * 2]
        
        energy = torch.tanh(self.attn(torch.cat((hidden, encoder_outputs), dim = 2))) 
        
        #energy = [batch size, src len, dec hid dim]

        attention = self.v(energy).squeeze(2)
        
        #attention = [batch size, src len]
        
        attention = attention.masked_fill(mask == 0, -1e10)
        
        return F.softmax(attention, dim = 1)

In [ ]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, enc_hid_dim, dec_hid_dim, dropout, attention):
        super().__init__()

        self.output_dim = output_dim
        self.attention = attention
        
        self.embedding = nn.Embedding(output_dim, emb_dim)
        
        self.rnn = nn.GRU((enc_hid_dim * 2) + emb_dim, dec_hid_dim)
        
        self.fc_out = nn.Linear((enc_hid_dim * 2) + dec_hid_dim + emb_dim, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input, hidden, encoder_outputs, mask):
             
        #input = [batch size]
        #hidden = [batch size, dec hid dim]
        #encoder_outputs = [src len, batch size, enc hid dim * 2]
        #mask = [batch size, src len]
        
        input = input.unsqueeze(0)
        
        #input = [1, batch size]
        
        embedded = self.dropout(self.embedding(input))
        
        #embedded = [1, batch size, emb dim]
        
        a = self.attention(hidden, encoder_outputs, mask)
                
        #a = [batch size, src len]
        
        a = a.unsqueeze(1)
        
        #a = [batch size, 1, src len]
        
        encoder_outputs = encoder_outputs.permute(1, 0, 2)
        
        #encoder_outputs = [batch size, src len, enc hid dim * 2]
        
        weighted = torch.bmm(a, encoder_outputs)
        
        #weighted = [batch size, 1, enc hid dim * 2]
        
        weighted = weighted.permute(1, 0, 2)
        
        #weighted = [1, batch size, enc hid dim * 2]
        
        rnn_input = torch.cat((embedded, weighted), dim = 2)
        
        #rnn_input = [1, batch size, (enc hid dim * 2) + emb dim]
            
        output, hidden = self.rnn(rnn_input, hidden.unsqueeze(0))
        
        #output = [seq len, batch size, dec hid dim * n directions]
        #hidden = [n layers * n directions, batch size, dec hid dim]
        
        #seq len, n layers and n directions will always be 1 in this decoder, therefore:
        #output = [1, batch size, dec hid dim]
        #hidden = [1, batch size, dec hid dim]
        #this also means that output == hidden
        assert (output == hidden).all()
        
        embedded = embedded.squeeze(0)
        output = output.squeeze(0)
        weighted = weighted.squeeze(0)
        
        prediction = self.fc_out(torch.cat((output, weighted, embedded), dim = 1))
        
        #prediction = [batch size, output dim]
        
        return prediction, hidden.squeeze(0), a.squeeze(1)

In [ ]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, src_pad_idx, device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.src_pad_idx = src_pad_idx
        self.device = device
        
    def create_mask(self, src):
        mask = (src != self.src_pad_idx).permute(1, 0)
        return mask
        
    def forward(self, src, src_len, trg, teacher_forcing_ratio = 0.5):
        
        #src = [src len, batch size]
        #src_len = [batch size]
        #trg = [trg len, batch size]
        #teacher_forcing_ratio is probability to use teacher forcing
        #e.g. if teacher_forcing_ratio is 0.75 we use teacher forcing 75% of the time
                    
        batch_size = src.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        
        #tensor to store decoder outputs
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
        
        #encoder_outputs is all hidden states of the input sequence, back and forwards
        #hidden is the final forward and backward hidden states, passed through a linear layer
        encoder_outputs, hidden = self.encoder(src, src_len)
                
        #first input to the decoder is the <sos> tokens
        input = trg[0,:]
        
        mask = self.create_mask(src)

        #mask = [batch size, src len]
                
        for t in range(1, trg_len):
            
            #insert input token embedding, previous hidden state, all encoder hidden states 
            #  and mask
            #receive output tensor (predictions) and new hidden state
            output, hidden, _ = self.decoder(input, hidden, encoder_outputs, mask)
            
            #place predictions in a tensor holding predictions for each token
            outputs[t] = output
            
            #decide if we are going to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio
            
            #get the highest predicted token from our predictions
            top1 = output.argmax(1) 
            
            #if teacher forcing, use actual next token as next input
            #if not, use predicted token
            input = trg[t] if teacher_force else top1
            
        return outputs

In [ ]:
INPUT_DIM = len(SRC.vocab)
OUTPUT_DIM = len(TRG.vocab)
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
ENC_HID_DIM = 512
DEC_HID_DIM = 512
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5
SRC_PAD_IDX = SRC.vocab.stoi[SRC.pad_token]

attn = Attention(ENC_HID_DIM, DEC_HID_DIM)
enc = Encoder(INPUT_DIM, ENC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, DEC_DROPOUT, attn)

model = Seq2Seq(enc, dec, SRC_PAD_IDX, device).to(device)

In [ ]:
def init_weights(m):
    for name, param in m.named_parameters():
        if 'weight' in name:
            nn.init.normal_(param.data, mean=0, std=0.01)
        else:
            nn.init.constant_(param.data, 0)
            
model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(7853, 256)
    (rnn): GRU(256, 512, bidirectional=True)
    (fc): Linear(in_features=1024, out_features=512, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (attention): Attention(
      (attn): Linear(in_features=1536, out_features=512, bias=True)
      (v): Linear(in_features=512, out_features=1, bias=False)
    )
    (embedding): Embedding(5893, 256)
    (rnn): GRU(1280, 512)
    (fc_out): Linear(in_features=1792, out_features=5893, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 20,518,405 trainable parameters


In [ ]:
optimizer = optim.Adam(model.parameters())

In [ ]:
TRG_PAD_IDX = TRG.vocab.stoi[TRG.pad_token]
criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)

### Train func

In [ ]:
def train(model, iterator, optimizer, criterion, clip):
    model.train()
    epoch_loss = 0
    
    for i, batch in enumerate(iterator):
        src, src_len = batch.src
        trg = batch.trg
        
        optimizer.zero_grad()
        
        output = model(src, src_len, trg)
        
        #trg = [trg len, batch size]
        #output = [trg len, batch size, output dim]
        
        output_dim = output.shape[-1]
        output = output[1:].view(-1, output_dim)
        trg = trg[1:].view(-1)
        
        #trg = [(trg len - 1) * batch size]
        #output = [(trg len - 1) * batch size, output dim]
        
        loss = criterion(output, trg)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [ ]:
def evaluate(model, iterator, criterion):
    model.eval()
    epoch_loss = 0
    
    with torch.no_grad():
        for i, batch in enumerate(iterator):
            src, src_len = batch.src
            trg = batch.trg

            output = model(src, src_len, trg, 0) #turn off teacher forcing
            
            #trg = [trg len, batch size]
            #output = [trg len, batch size, output dim]

            output_dim = output.shape[-1]
            
            output = output[1:].view(-1, output_dim)
            trg = trg[1:].view(-1)
            #trg = [(trg len - 1) * batch size]
            #output = [(trg len - 1) * batch size, output dim]

            loss = criterion(output, trg)
            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [ ]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

### Train

In [ ]:
for batch in train_iterator:
  break
batch.src[0].shape

torch.Size([11, 128])

In [ ]:
TRG.vocab.stoi["<sos>"], TRG.vocab.stoi["<eos>"], TRG.vocab.stoi["<pad>"], TRG.vocab.itos[0]

(2, 3, 1, '<unk>')

In [ ]:
N_EPOCHS = 5
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)
    valid_loss = 0
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut4-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

Epoch: 01 | Time: 0m 51s
	Train Loss: 2.358 | Train PPL:  10.565
	 Val. Loss: 0.000 |  Val. PPL:   1.000


KeyboardInterrupt: ignored

In [ ]:
# Epoch: 01 | Time: 0m 57s
# 	Train Loss: 4.716 | Train PPL: 111.752
# 	 Val. Loss: 4.335 |  Val. PPL:  76.338
# Epoch: 02 | Time: 0m 57s
# 	Train Loss: 3.406 | Train PPL:  30.133
# 	 Val. Loss: 3.571 |  Val. PPL:  35.548
# Epoch: 03 | Time: 0m 56s
# 	Train Loss: 2.705 | Train PPL:  14.957
# 	 Val. Loss: 3.321 |  Val. PPL:  27.686
# Epoch: 04 | Time: 0m 57s
# 	Train Loss: 2.288 | Train PPL:   9.858
# 	 Val. Loss: 3.225 |  Val. PPL:  25.160
# Epoch: 05 | Time: 0m 57s
# 	Train Loss: 2.003 | Train PPL:   7.414
# 	 Val. Loss: 3.141 |  Val. PPL:  23.123
# Epoch: 06 | Time: 0m 57s
# 	Train Loss: 1.755 | Train PPL:   5.786
# 	 Val. Loss: 3.186 |  Val. PPL:  24.185

# My Section

## Setup

In [8]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [52]:
# spacy_de = spacy.load('de_core_news_sm')
# def tokenize_de(text):
#     return [tok.text for tok in spacy_de.tokenizer(text)]

spacy_fr = spacy.load('fr_core_news_sm')
def tokenize_fr(text):
    return [tok.text for tok in spacy_fr.tokenizer(text)]

spacy_en = spacy.load('en_core_web_sm')
def tokenize_en(text):
    return [tok.text for tok in spacy_en.tokenizer(text)][::-1]

## Data

In [53]:
with open('/content/gdrive/MyDrive/Colab Notebooks/eaai24/Datasets/enfr_160kpairs_2k5-freq-words.pkl', 'rb') as f:
  data = pickle.load(f)
data[0], len(data)

({'en': 'Madam President, on a point of order.',
  'fr': "Madame la Présidente, c'est une motion de procédure."},
 158629)

In [54]:
EN_SRC = Field(tokenize = tokenize_en, 
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = True, 
            include_lengths = True)

FR_TRG = Field(tokenize = tokenize_fr, 
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = True)

In [55]:
train_len = 64000
valid_len = 3200
test_len = 3200

train_pt = train_len
valid_pt = train_pt + valid_len
test_pt = valid_pt + test_len

In [56]:
data_set = [[pair['en'], pair['fr']] for pair in data]
FIELDS = [('src', EN_SRC), ('trg', FR_TRG)]
train_examples = list(map(lambda x: Example.fromlist(list(x), fields=FIELDS), data_set[: train_pt]))
valid_examples = list(map(lambda x: Example.fromlist(list(x), fields=FIELDS), data_set[train_pt : valid_pt]))
test_examples = list(map(lambda x: Example.fromlist(list(x), fields=FIELDS), data_set[valid_pt : test_pt]))

In [57]:
train_dt = Dataset(train_examples, fields=FIELDS)
valid_dt = Dataset(valid_examples, fields=FIELDS)
test_dt = Dataset(test_examples, fields=FIELDS)
train_dt, valid_dt, test_dt

(<torchtext.legacy.data.dataset.Dataset at 0x7efef7fb1df0>,
 <torchtext.legacy.data.dataset.Dataset at 0x7efef7fb1040>)

In [30]:
valid_dt.src

<generator object Dataset.__getattr__ at 0x7eff843d97b0>

In [58]:
EN_SRC.build_vocab(train_dt, min_freq = 2)
FR_TRG.build_vocab(train_dt, min_freq = 2)

In [59]:
BATCH_SIZE = 32
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator1, valid_iterator1, test_iterator1 = BucketIterator.splits(
    (train_dt, valid_dt, test_dt),
     batch_size = BATCH_SIZE,
     sort_within_batch = True,
     sort_key = lambda x : len(x.src),
     device = device)

In [60]:
for i, batch1 in enumerate(train_iterator1):
  print(batch1.src[0].shape, batch1.trg.shape)
  if i==5: break
batch1.fields

torch.Size([15, 32]) torch.Size([24, 32])
torch.Size([11, 32]) torch.Size([16, 32])
torch.Size([23, 32]) torch.Size([32, 32])
torch.Size([11, 32]) torch.Size([16, 32])
torch.Size([16, 32]) torch.Size([20, 32])
torch.Size([15, 32]) torch.Size([22, 32])


dict_keys(['src', 'trg'])

In [33]:
batch1.src[0].shape, batch1.trg.shape

(torch.Size([15, 32]), torch.Size([22, 32]))

In [ ]:
batch1.src, batch1.trg

In [61]:
src_sent = []
trg_sent = []
for i in batch1.src[0][: , 0]:
  src_sent.append(EN_SRC.vocab.itos[i])
for i in batch1.trg[:, 0]:
  trg_sent.append(FR_TRG.vocab.itos[i])
print(' '.join(src_sent), '\n', ' '.join(trg_sent))

<sos> . rules market than more matters rights fundamental of charter the , certainly <eos> 
 <sos> la charte des droits fondamentaux vaut certainement davantage que les règles du marché . <eos> <pad> <pad> <pad> <pad> <pad> <pad>


In [62]:
EN_SRC.vocab.itos[0], EN_SRC.vocab.itos[1], EN_SRC.vocab.itos[2], EN_SRC.vocab.itos[3]

('<unk>', '<pad>', '<sos>', '<eos>')

In [47]:
batch1.src[0][:, 0]

tensor([  2, 380,   6,   5, 636,   9, 254, 127, 485,  67, 180, 200, 343,   4,
          3], device='cuda:0')

## Model

### Encoder

In [63]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, enc_hid_dim, dec_hid_dim, dropout):
        super().__init__()
        
        self.embedding = nn.Embedding(input_dim, emb_dim)
        
        self.rnn = nn.GRU(emb_dim, enc_hid_dim, bidirectional = True)
        
        self.fc = nn.Linear(enc_hid_dim * 2, dec_hid_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src, src_len):
        
        #src = [src len, batch size]
        #src_len = [batch size]
        
        embedded = self.dropout(self.embedding(src))
        
        #embedded = [src len, batch size, emb dim]
                
        #need to explicitly put lengths on cpu!
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, src_len.to('cpu'))
                
        packed_outputs, hidden = self.rnn(packed_embedded)
                                 
        #packed_outputs is a packed sequence containing all hidden states
        #hidden is now from the final non-padded element in the batch
            
        outputs, _ = nn.utils.rnn.pad_packed_sequence(packed_outputs) 
            
        #outputs is now a non-packed sequence, all hidden states obtained
        #  when the input is a pad token are all zeros
            
        #outputs = [src len, batch size, hid dim * num directions]
        #hidden = [n layers * num directions, batch size, hid dim]
        
        #hidden is stacked [forward_1, backward_1, forward_2, backward_2, ...]
        #outputs are always from the last layer
        
        #hidden [-2, :, : ] is the last of the forwards RNN 
        #hidden [-1, :, : ] is the last of the backwards RNN
        
        #initial decoder hidden is final hidden state of the forwards and backwards 
        #  encoder RNNs fed through a linear layer
        hidden = torch.tanh(self.fc(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1)))
        
        #outputs = [src len, batch size, enc hid dim * 2]
        #hidden = [batch size, dec hid dim]
        
        return outputs, hidden

### Attn

In [64]:
class Attention(nn.Module):
    def __init__(self, enc_hid_dim, dec_hid_dim):
        super().__init__()
        
        self.attn = nn.Linear((enc_hid_dim * 2) + dec_hid_dim, dec_hid_dim)
        self.v = nn.Linear(dec_hid_dim, 1, bias = False)
        
    def forward(self, hidden, encoder_outputs, mask):
        
        #hidden = [batch size, dec hid dim]
        #encoder_outputs = [src len, batch size, enc hid dim * 2]
        
        batch_size = encoder_outputs.shape[1]
        src_len = encoder_outputs.shape[0]
        
        #repeat decoder hidden state src_len times
        hidden = hidden.unsqueeze(1).repeat(1, src_len, 1)
  
        encoder_outputs = encoder_outputs.permute(1, 0, 2)
        
        #hidden = [batch size, src len, dec hid dim]
        #encoder_outputs = [batch size, src len, enc hid dim * 2]
        
        energy = torch.tanh(self.attn(torch.cat((hidden, encoder_outputs), dim = 2))) 
        
        #energy = [batch size, src len, dec hid dim]

        attention = self.v(energy).squeeze(2)
        
        #attention = [batch size, src len]
        
        attention = attention.masked_fill(mask == 0, -1e10)
        
        return F.softmax(attention, dim = 1)

### Decoder

In [65]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, enc_hid_dim, dec_hid_dim, dropout, attention):
        super().__init__()

        self.output_dim = output_dim
        self.attention = attention
        
        self.embedding = nn.Embedding(output_dim, emb_dim)
        
        self.rnn = nn.GRU((enc_hid_dim * 2) + emb_dim, dec_hid_dim)
        
        self.fc_out = nn.Linear((enc_hid_dim * 2) + dec_hid_dim + emb_dim, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input, hidden, encoder_outputs, mask):
             
        #input = [batch size]
        #hidden = [batch size, dec hid dim]
        #encoder_outputs = [src len, batch size, enc hid dim * 2]
        #mask = [batch size, src len]
        
        input = input.unsqueeze(0)
        
        #input = [1, batch size]
        
        embedded = self.dropout(self.embedding(input))
        
        #embedded = [1, batch size, emb dim]
        
        a = self.attention(hidden, encoder_outputs, mask)
                
        #a = [batch size, src len]
        
        a = a.unsqueeze(1)
        
        #a = [batch size, 1, src len]
        
        encoder_outputs = encoder_outputs.permute(1, 0, 2)
        
        #encoder_outputs = [batch size, src len, enc hid dim * 2]
        
        weighted = torch.bmm(a, encoder_outputs)
        
        #weighted = [batch size, 1, enc hid dim * 2]
        
        weighted = weighted.permute(1, 0, 2)
        
        #weighted = [1, batch size, enc hid dim * 2]
        
        rnn_input = torch.cat((embedded, weighted), dim = 2)
        
        #rnn_input = [1, batch size, (enc hid dim * 2) + emb dim]
            
        output, hidden = self.rnn(rnn_input, hidden.unsqueeze(0))
        
        #output = [seq len, batch size, dec hid dim * n directions]
        #hidden = [n layers * n directions, batch size, dec hid dim]
        
        #seq len, n layers and n directions will always be 1 in this decoder, therefore:
        #output = [1, batch size, dec hid dim]
        #hidden = [1, batch size, dec hid dim]
        #this also means that output == hidden
        assert (output == hidden).all()
        
        embedded = embedded.squeeze(0)
        output = output.squeeze(0)
        weighted = weighted.squeeze(0)
        
        prediction = self.fc_out(torch.cat((output, weighted, embedded), dim = 1))
        
        #prediction = [batch size, output dim]
        
        return prediction, hidden.squeeze(0), a.squeeze(1)

### Seq2Seq

In [66]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, src_pad_idx, device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.src_pad_idx = src_pad_idx
        self.device = device
        
    def create_mask(self, src):
        mask = (src != self.src_pad_idx).permute(1, 0)
        return mask
        
    def forward(self, src, src_len, trg, teacher_forcing_ratio = 0.5):
        
        #src = [src len, batch size]
        #src_len = [batch size]
        #trg = [trg len, batch size]
        #teacher_forcing_ratio is probability to use teacher forcing
        #e.g. if teacher_forcing_ratio is 0.75 we use teacher forcing 75% of the time
                    
        batch_size = src.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        
        #tensor to store decoder outputs
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
        
        #encoder_outputs is all hidden states of the input sequence, back and forwards
        #hidden is the final forward and backward hidden states, passed through a linear layer
        encoder_outputs, hidden = self.encoder(src, src_len)
                
        #first input to the decoder is the <sos> tokens
        input = trg[0,:]
        
        mask = self.create_mask(src)

        #mask = [batch size, src len]
                
        for t in range(1, trg_len):
            
            #insert input token embedding, previous hidden state, all encoder hidden states 
            #  and mask
            #receive output tensor (predictions) and new hidden state
            output, hidden, _ = self.decoder(input, hidden, encoder_outputs, mask)
            
            #place predictions in a tensor holding predictions for each token
            outputs[t] = output
            
            #decide if we are going to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio
            
            #get the highest predicted token from our predictions
            top1 = output.argmax(1) 
            
            #if teacher forcing, use actual next token as next input
            #if not, use predicted token
            input = trg[t] if teacher_force else top1
            
        return outputs
    
    def compute_loss(self, src, src_len, trg, criterion, teacher_forcing_ratio = 0.5):
        output = self.forward(src, src_len, trg, teacher_forcing_ratio = teacher_forcing_ratio) #output = [trg len, batch size, output dim]

        output_dim = output.shape[-1]
        output_ = output[1:].view(-1, output_dim)  #output = [(trg len - 1) * batch size, output dim]
        trg = trg[1:].view(-1)  #trg = [(trg len - 1) * batch size]

        loss = criterion(output_, trg)
        return loss, output

### Pivot model

In [ ]:
class PivotSeq2Seq(nn.Module):
    def __init__(self, model1, model2, src_pad_idx, device):
        super().__init__()
        
        self.model1 = model1
        self.model2 = model2
        self.src_pad_idx = src_pad_idx
        self.device = device
        
    def create_mask(self, src):
        mask = (src != self.src_pad_idx).permute(1, 0)
        return mask
        
    def forward(self, src, src_len, trg, teacher_forcing_ratio = 0.5):
        
        loss1, output1 = self.model1.compute_loss(src, src_len, trg, criterion, teacher_forcing_ratio = teacher_forcing_ratio)
        # output1 = [trg len, batch size, output dim]
        
        # HOW TO CONVERT output1 to src shape???
        
        #src = [src len, batch size]
        #src_len = [batch size]
        #trg = [trg len, batch size]
        return outputs
    
    def compute_loss(self, src, src_len, trg, criterion, teacher_forcing_ratio = 0.5):
        output = self.forward(src, src_len, trg, teacher_forcing_ratio = teacher_forcing_ratio) #output = [trg len, batch size, output dim]

        output_dim = output.shape[-1]
        output = output[1:].view(-1, output_dim)  #output = [(trg len - 1) * batch size, output dim]
        trg = trg[1:].view(-1)  #trg = [(trg len - 1) * batch size]

        loss = criterion(output, trg)
        return loss, output

## Train func

In [78]:
def train(model, iterator, optimizer, criterion, clip):
  model.train()
  epoch_loss = 0.0
  
  for batch in tqdm(iterator):
    src, src_len = batch.src
    trg = batch.trg # trg = [trg len, batch size]
    
    optimizer.zero_grad()

    ####### move to seq2seq model #######
    # output = model(src, src_len, trg) #output = [trg len, batch size, output dim]
    # output_dim = output.shape[-1]
    # output = output[1:].view(-1, output_dim)  #output = [(trg len - 1) * batch size, output dim]
    # trg = trg[1:].view(-1)  #trg = [(trg len - 1) * batch size]
    # loss = criterion(output, trg)
    ####### END MOVING PART #####
    loss, output = model.compute_loss(src, src_len, trg, criterion)

    loss.backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
    optimizer.step()
    
    epoch_loss += loss.item()
      
  return epoch_loss / len(iterator)

In [79]:
def evaluate(model, iterator, criterion):
  model.eval()
  epoch_loss = 0.0
  
  with torch.no_grad():
    for batch in tqdm(iterator):
      src, src_len = batch.src
      trg = batch.trg #trg = [trg len, batch size]

      ####### move to seq2seq model #######      
      # output = model(src, src_len, trg, 0)  #output = [trg len, batch size, output dim]
      # output_dim = output.shape[-1]
      # output = output[1:].view(-1, output_dim)  #output = [(trg len - 1) * batch size, output dim]
      # trg = trg[1:].view(-1)  #trg = [(trg len - 1) * batch size]
      # loss = criterion(output, trg)
      ####### END MOVING PART #####
      
      # turn off teacher forcing
      loss, output = model.compute_loss(src, src_len, trg, criterion, 0)
      epoch_loss += loss.item()
      
  return epoch_loss / len(iterator)

In [69]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [70]:
def update_trainlog(data: list, filename: str='/content/gdrive/MyDrive/Colab Notebooks/eaai24/training_log.txt'):
    ''' Update training log w/ new losses
    Args:
        data (List): a list of infor for many epochs as tuple, each tuple has model_name, loss, etc.
        filename (String): path + file_name
    Return:
        None: new data is appended into train-log
    '''
    with open(filename, 'a') as f: # save
        for epoch in data:
            f.write(','.join(epoch))
            f.write("\n")
    print('update_trainlog SUCCESS')
    return []

## Train

In [80]:
INPUT_DIM = len(EN_SRC.vocab)
OUTPUT_DIM = len(FR_TRG.vocab)
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
ENC_HID_DIM = 512
DEC_HID_DIM = 512
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5
LR = 0.001
SRC_PAD_IDX = EN_SRC.vocab.stoi[EN_SRC.pad_token]

attn1 = Attention(ENC_HID_DIM, DEC_HID_DIM)
enc1 = Encoder(INPUT_DIM, ENC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, ENC_DROPOUT)
dec1 = Decoder(OUTPUT_DIM, DEC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, DEC_DROPOUT, attn1)

model1 = Seq2Seq(enc1, dec1, SRC_PAD_IDX, device).to(device)

In [81]:
def init_weights(m):
    for name, param in m.named_parameters():
        if 'weight' in name:
            nn.init.normal_(param.data, mean=0, std=0.01)
        else:
            nn.init.constant_(param.data, 0)
            
model1.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(2421, 256)
    (rnn): GRU(256, 512, bidirectional=True)
    (fc): Linear(in_features=1024, out_features=512, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (attention): Attention(
      (attn): Linear(in_features=1536, out_features=512, bias=True)
      (v): Linear(in_features=512, out_features=1, bias=False)
    )
    (embedding): Embedding(2491, 256)
    (rnn): GRU(1280, 512)
    (fc_out): Linear(in_features=1792, out_features=2491, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [82]:
optimizer1 = optim.Adam(model1.parameters(), lr=LR)

In [83]:
criterion1 = nn.CrossEntropyLoss(ignore_index = FR_TRG.vocab.stoi[FR_TRG.pad_token])

In [84]:
N_EPOCHS = 5
CLIP = 1
best_valid_loss = float('inf')
best_train_loss = float('inf')
model_name = 'attn_en-fr_32k_160kset_inverse.pt'
train_log = []

In [85]:
for epoch in range(N_EPOCHS):
  start_time = time.time()
  train_loss = train(model1, train_iterator1, optimizer1, criterion1, CLIP)
  valid_loss = evaluate(model1, valid_iterator1, criterion1)

  end_time = time.time()
  epoch_mins, epoch_secs = epoch_time(start_time, end_time)
  
  epoch_info = [model_name, str(LR), str(BATCH_SIZE), str(ENC_HID_DIM), 'no-num_layers', str(ENC_DROPOUT), str(DEC_DROPOUT), str(epoch), str(N_EPOCHS), str(train_loss), str(valid_loss)]
  train_log.append(epoch_info)

  if valid_loss < best_valid_loss:
    best_valid_loss = valid_loss
    torch.save(model1.state_dict(), f'/content/gdrive/MyDrive/Colab Notebooks/eaai24/valid_{model_name}')
    print('SAVED MODEL - valid')
  if train_loss < best_train_loss:
    best_train_loss = train_loss
    torch.save(model1.state_dict(), f'/content/gdrive/MyDrive/Colab Notebooks/eaai24/{model_name}')
    print('SAVED MODEL')
    train_log = update_trainlog(train_log)
  
  print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
  print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
  print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

100%|██████████| 100/100 [00:02<00:00, 39.91it/s]


SAVED MODEL - valid
SAVED MODEL
update_trainlog SUCCESS
Epoch: 01 | Time: 2m 38s
	Train Loss: 3.680 | Train PPL:  39.655
	 Val. Loss: 3.191 |  Val. PPL:  24.315


100%|██████████| 100/100 [00:02<00:00, 42.46it/s]


SAVED MODEL - valid
SAVED MODEL
update_trainlog SUCCESS
Epoch: 02 | Time: 2m 38s
	Train Loss: 2.576 | Train PPL:  13.142
	 Val. Loss: 2.998 |  Val. PPL:  20.052


100%|██████████| 100/100 [00:02<00:00, 42.49it/s]


SAVED MODEL - valid
SAVED MODEL
update_trainlog SUCCESS
Epoch: 03 | Time: 2m 40s
	Train Loss: 2.267 | Train PPL:   9.653
	 Val. Loss: 2.917 |  Val. PPL:  18.481


100%|██████████| 100/100 [00:02<00:00, 42.61it/s]


SAVED MODEL
update_trainlog SUCCESS
Epoch: 04 | Time: 2m 40s
	Train Loss: 2.081 | Train PPL:   8.013
	 Val. Loss: 2.936 |  Val. PPL:  18.849


100%|██████████| 100/100 [00:03<00:00, 33.15it/s]


SAVED MODEL
update_trainlog SUCCESS
Epoch: 05 | Time: 2m 39s
	Train Loss: 1.954 | Train PPL:   7.055
	 Val. Loss: 3.029 |  Val. PPL:  20.687


In [86]:
# Continue training
for epoch in range(N_EPOCHS, N_EPOCHS+5):
  start_time = time.time()
  train_loss = train(model1, train_iterator1, optimizer1, criterion1, CLIP)
  valid_loss = evaluate(model1, valid_iterator1, criterion1)

  end_time = time.time()
  epoch_mins, epoch_secs = epoch_time(start_time, end_time)
  
  epoch_info = [model_name, str(LR), str(BATCH_SIZE), str(ENC_HID_DIM), 'no-num_layers', str(ENC_DROPOUT), str(DEC_DROPOUT), str(epoch), str(N_EPOCHS), str(train_loss), str(valid_loss)]
  train_log.append(epoch_info)

  if valid_loss < best_valid_loss or train_loss < best_train_loss:
    best_valid_loss = valid_loss
    best_train_loss = train_loss
    torch.save(model1.state_dict(), f'/content/gdrive/MyDrive/Colab Notebooks/eaai24/{model_name}')
    print('SAVED MODEL')
    train_log = update_trainlog(train_log)
  
  print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
  print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
  print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

100%|██████████| 100/100 [00:02<00:00, 43.34it/s]


SAVED MODEL
update_trainlog SUCCESS
Epoch: 06 | Time: 2m 46s
	Train Loss: 1.850 | Train PPL:   6.359
	 Val. Loss: 3.042 |  Val. PPL:  20.948


100%|██████████| 100/100 [00:02<00:00, 42.72it/s]


SAVED MODEL
update_trainlog SUCCESS
Epoch: 07 | Time: 2m 47s
	Train Loss: 1.780 | Train PPL:   5.929
	 Val. Loss: 3.105 |  Val. PPL:  22.300


100%|██████████| 100/100 [00:02<00:00, 42.36it/s]


SAVED MODEL
update_trainlog SUCCESS
Epoch: 08 | Time: 2m 54s
	Train Loss: 1.715 | Train PPL:   5.557
	 Val. Loss: 3.156 |  Val. PPL:  23.482


100%|██████████| 100/100 [00:02<00:00, 33.89it/s]


SAVED MODEL
update_trainlog SUCCESS
Epoch: 09 | Time: 2m 51s
	Train Loss: 1.675 | Train PPL:   5.338
	 Val. Loss: 3.165 |  Val. PPL:  23.696


100%|██████████| 100/100 [00:02<00:00, 43.90it/s]


SAVED MODEL
update_trainlog SUCCESS
Epoch: 10 | Time: 2m 38s
	Train Loss: 1.624 | Train PPL:   5.075
	 Val. Loss: 3.176 |  Val. PPL:  23.956


## Eval

In [ ]:
# FIELDS = [('src', EN_SRC), ('trg', FR_TRG)]
# start_idx = train_len + valid_len
# end_idx = start_idx + 6400
# test_examples = list(map(lambda x: Example.fromlist(list(x), fields=FIELDS), data_set[start_idx : end_idx]))
# test_dt = Dataset(test_examples, fields=FIELDS)
# test_iterator1 = BucketIterator(
#     test_dt,
#      batch_size = BATCH_SIZE,
#      sort_within_batch = True,
#      sort_key = lambda x : len(x.src),
#      device = device)

In [87]:
for batch in test_iterator1:
  break
batch.fields

dict_keys(['src', 'trg'])

In [96]:
attn = Attention(ENC_HID_DIM, DEC_HID_DIM)
enc = Encoder(INPUT_DIM, ENC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, DEC_DROPOUT, attn)

model_infer = Seq2Seq(enc, dec, SRC_PAD_IDX, device).to(device)

In [103]:
model_infer.load_state_dict(torch.load(f'/content/gdrive/MyDrive/Colab Notebooks/eaai24/{model_name}'))
test_loss = evaluate(model_infer, test_iterator1, criterion1)
print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')

100%|██████████| 100/100 [00:02<00:00, 33.82it/s]

| Test Loss: 2.982 | Test PPL:  19.719 |


## Inference

In [98]:
def translate_sentence(sentence, src_field, trg_field, model, device, max_len = 50):
  model.eval()
  if isinstance(sentence, str):
    nlp = spacy.load('en_core_web_sm')
    tokens = [token.text.lower() for token in nlp(sentence)]
  else:
    tokens = [token.lower() for token in sentence]
  tokens = [src_field.init_token] + tokens + [src_field.eos_token]       
  src_indexes = [src_field.vocab.stoi[token] for token in tokens]    
  src_tensor = torch.LongTensor(src_indexes).unsqueeze(1).to(device)
  src_len = torch.LongTensor([len(src_indexes)])    
  with torch.no_grad():
    encoder_outputs, hidden = model.encoder(src_tensor, src_len)

  mask = model.create_mask(src_tensor)
  trg_indexes = [trg_field.vocab.stoi[trg_field.init_token]]
  attentions = torch.zeros(max_len, 1, len(src_indexes)).to(device)
  for i in range(max_len):
    trg_tensor = torch.LongTensor([trg_indexes[-1]]).to(device)
    with torch.no_grad():
      output, hidden, attention = model.decoder(trg_tensor, hidden, encoder_outputs, mask)

    attentions[i] = attention
    pred_token = output.argmax(1).item()
    trg_indexes.append(pred_token)
    if pred_token == trg_field.vocab.stoi[trg_field.eos_token]:
      break
  
  trg_tokens = [trg_field.vocab.itos[i] for i in trg_indexes]
  return trg_tokens[1:], attentions[:len(trg_tokens)-1]

In [99]:
example_idx = 12
src = vars(test_dt.examples[example_idx])['src']
trg = vars(test_dt.examples[example_idx])['trg']
print(f'src = {src}')
print(f'trg = {trg}')

src = ['.', 'closed', 'is', 'debate', 'the']
trg = ['le', 'débat', 'est', 'clos', '.']


In [100]:
translation, attention = translate_sentence(src, EN_SRC, FR_TRG, model_infer, device)
print(f'predicted trg = {translation}')

predicted trg = ['le', 'débat', 'est', 'clos', '.', '<eos>']


## BLEU

In [101]:
from torchtext.data.metrics import bleu_score
def calculate_bleu(data, src_field, trg_field, model, device, max_len = 50):
  trgs = []
  pred_trgs = []
  for datum in tqdm(data):
    src = vars(datum)['src']
    trg = vars(datum)['trg']
    pred_trg, _ = translate_sentence(src, src_field, trg_field, model, device, max_len)
    #cut off <eos> token
    pred_trg = pred_trg[:-1]
    pred_trgs.append(pred_trg)
    trgs.append([trg])        
  return bleu_score(pred_trgs, trgs)

In [104]:
bleu_score = calculate_bleu(test_dt, EN_SRC, FR_TRG, model_infer, device)
print(f'BLEU score = {bleu_score*100:.2f}')

100%|██████████| 3200/3200 [00:50<00:00, 63.53it/s]


BLEU score = 32.15


* attn_en-fr_32k.pt: BLEU = 12.65
* attn_en-fr_32k_160kset_inverse.pt: BLEU = 32.15
* valid_attn_en-fr_32k_160kset_inverse.pt: BLEU = ?


# End